In [1]:
from gym.spaces import Tuple, Discrete, Box
import ray
from ray import tune
from ray.tune.registry import register_env
from ray.rllib.env.multi_agent_env import MultiAgentEnv
import numpy as np
from ray.rllib.utils import check_env
from typing import Optional


class TestEnv(MultiAgentEnv):
    def __init__(self, config):
        # Mosse 
        self.MOVES = ['mossa1', 'mossa2','mossa3']
        # Questa è la truncation cosi esce per non girare all'infinito
        self.NUM_ITERS = 20
        # Mappa che in base all'azione eseguita mi da costo, impatto, ecc dell'azione
        self.REWARD_MAP = {
            'mossa1': (1, 0, 0),
            'mossa2': (2, 0, 0),
            'mossa3': (3, 0, 0)
        }
        # per la funzione di reward
        self.wt = 0.5
        self.wc = 0.5
        self.wi = 0.5
        self.tMax = 100
        self.cMax = 100
        self.possible_agents = ['attaccante','difensore']
        self._agent_ids = set(self.possible_agents)
        self._action_spaces = {agent: Discrete(3) for agent in self.possible_agents}
        self._observation_spaces = {
            agent:dict(
                {
                    'observation': Box(low=0,high=1,shape=(2,),dtype=bool),
                }
            )
            for agent in self.possible_agents
        }

    def step(self,allAction):
        if (
            self.terminations[self.agent_selection]
            or self.truncations[self.agent_selection]
        ):
            #self._was_dead_step(action)
            print('ECCOOOO:',self.agents)
            return

        for agent in self.agents:
            #agent = self.agent_selection
            print('Agente in azione:',agent)
            action = allAction[agent]
            reward = self.REWARD_MAP[self.MOVES[action]]
            valReward = -self.wt*(reward[0]/self.tMax)-self.wc*(reward[1]/self.cMax)-self.wi*(1) 
            #rewardInv = -valReward
            print('Reward:',valReward)
            if agent == 'attaccante':
                self.rewards[self.agents[0]], self.rewards[self.agents[1]] = (valReward,0)
            else:
                self.rewards[self.agents[0]], self.rewards[self.agents[1]] = (0,valReward)
            self.num_moves += 1
            # The truncations dictionary must be updated for all players.
            self.truncations = {
                agent: self.num_moves >= self.NUM_ITERS for agent in self.agents
            }
            #self.agent_selection = self._agent_selector.next()
            # Adds .rewards to ._cumulative_rewards
            #self._accumulate_rewards() """
        return Tuple(self._observation_spaces,self.rewards,self.terminations,self.truncations,self.infos)
        
    def reset(self,seed,options):
        self.agents = self.possible_agents[:]
        self.rewards = {agent: 0 for agent in self.agents}
        self._cumulative_rewards = {agent: 0 for agent in self.agents}      
        self.terminations = {agent: False for agent in self.agents}
        self.truncations = {agent: False for agent in self.agents}
        self.infos = {agent: {} for agent in self.agents}
        #self.state = {agent: NONE for agent in self.agents}
        #self.observations = {agent: NONE for agent in self.agents}
        # credo serve per arrestare
        self.num_moves = 0
        """
        Our agent_selector utility allows easy cyclic stepping through the agents list.
        """
        #self._agent_selector = agent_selector(self.agents)
        #self.agent_selection = self._agent_selector.reset()
        return {agent: Tuple(self._observation_spaces[agent]) for agent in range(self.agents) }
    
    def observation_space(self, agent):
        # gymnasium spaces are defined and documented here: https://gymnasium.farama.org/api/spaces/
        return self._observation_spaces[agent]
    def action_space(self, agent):
        return self._action_spaces[agent]
        


def env_creator(args):
    env = TestEnv(args)
    obs_space = env.observation_space
    act_space = env.action_space
    n_agents = len(args["agents"])
    obs_space = Tuple([obs_space for _ in range(n_agents)])
    act_space = Tuple([act_space for _ in range(n_agents)])
    grouping = {"group_1": [str(i) for i in range(n_agents)]}
    return env.with_agent_groups(grouping, obs_space=obs_space, act_space=act_space)


register_env("grouped_test", env_creator)
results = tune.run(
    "QMIX",
    stop={"timesteps_total": 5,},
    config={
        "mixer": "qmix",
        "env": "grouped_test",
        "env_config": {"agents": ["0", "1"]},
        "num_workers": 0,
        "timesteps_per_iteration": 2,
    },
)

2023-09-25 16:17:00,956	INFO util.py:159 -- Outdated packages:
  ipywidgets==6.0.0 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2023-09-25 16:17:01,584	INFO util.py:159 -- Outdated packages:
  ipywidgets==6.0.0 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2023-09-25 16:17:05,768	WARNING deprecation.py:50 -- DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!
2023-09-25 16:17:08,915	INFO worker.py:1633 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 
2023-09-25 16:17:11,072	INFO tune.py:228 -- Initializing Ray automatically. For cluster usage or custom Ray initialization, call `ray.init(...)` before `tune.run(...)`.
2023-09-25 16:17:11,076	INFO tune.py:645 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the

(pid=381351) DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!
(QMix pid=381351) 2023-09-25 16:17:17,436	WARNING deprecation.py:50 -- DeprecationWarning: `rllib/algorithms/qmix/` has been deprecated. Use `rllib_contrib/qmix/` instead. This will raise an error in the future!
(QMix pid=381351) 2023-09-25 16:17:17,436	WARNING deprecation.py:50 -- DeprecationWarning: `rllib/algorithms/simple_q/` has been deprecated. Use `rllib_contrib/simple_q/` instead. This will raise an error in the future!
(QMix pid=381351) 2023-09-25 16:17:17,436	WARNING algorithm_config.py:672 -- Cannot create QMixConfig from given `config_dict`! Property __stdout_file__ not supported.
2023-09-25 16:17:17,671	ERROR tune_controller.py:1502 -- Trial task failed for trial QMIX_grouped_test_37c50_00000
Traceback (most recent call last):
  File "/home/matteo/.local/lib/python3.10/site-packages/ray/air/execution/_internal/event_manager.py", line 110, in resolve_future
  

Trial name
QMIX_grouped_test_37c50_00000


(QMix pid=381351) Exception raised in creation task: The actor died because of an error raised in its creation task, ray::QMix.__init__() (pid=381351, ip=10.25.252.249, actor_id=9686c28365e483c3f85d6ae101000000, repr=QMix)
(QMix pid=381351)   File "/home/matteo/.local/lib/python3.10/site-packages/ray/rllib/utils/deprecation.py", line 106, in patched_init
(QMix pid=381351)     return obj_init(*args, **kwargs)
(QMix pid=381351)   File "/home/matteo/.local/lib/python3.10/site-packages/ray/rllib/utils/deprecation.py", line 106, in patched_init
(QMix pid=381351)     return obj_init(*args, **kwargs)
(QMix pid=381351)   File "/home/matteo/.local/lib/python3.10/site-packages/ray/rllib/algorithms/algorithm.py", line 517, in __init__
(QMix pid=381351)     super().__init__(
(QMix pid=381351)   File "/home/matteo/.local/lib/python3.10/site-packages/ray/tune/trainable/trainable.py", line 185, in __init__
(QMix pid=381351)     self.setup(copy.deepcopy(self.config))
(QMix pid=381351)   File "/home/ma

TuneError: ('Trials did not complete', [QMIX_grouped_test_37c50_00000])